# Assignment 1: Data Curation
Ankit Tandon

In [1]:
import json
import requests

In [2]:
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/{project}/{access-site}/{granularity}/{start}/{end}'

endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/{project}/{access}/{agent}/{granularity}/{start}/{end}'

In [3]:
# parameters for getting aggregated legacy view data 
# see: https://wikimedia.org/api/rest_v1/#!/Legacy_data/get_metrics_legacy_pagecounts_aggregate_project_access_site_granularity_start_end
params_legacy_desktop_site = {"project" : "en.wikipedia.org",
                 "access-site" : "desktop-site",
                 "granularity" : "monthly",
                 "start" : "2007120100", 
                # for end use 1st day of month following final month of data
                 "end" : "2016080100"
                    }
params_legacy_mobile = {"project" : "en.wikipedia.org",
                 "access-site" : "mobile-site",
                 "granularity" : "monthly",
                 "start" : "2007120100", 
                # for end use 1st day of month following final month of data
                 "end" : "2016080100"
                    }

# parameters for getting aggregated current standard pageview data
# see: https://wikimedia.org/api/rest_v1/#!/Pageviews_data/get_metrics_pageviews_aggregate_project_access_agent_granularity_start_end
params_pageviews_desktop = {"project" : "en.wikipedia.org",
                    "access" : "desktop",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                    "end" : '2018100100'
                        }
params_pageviews_mobile_web = {"project" : "en.wikipedia.org",
                    "access" : "mobile-web",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                    "end" : '2018100100'
                        }
params_pageviews_mobile_app = {"project" : "en.wikipedia.org",
                    "access" : "mobile-app",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                    "end" : '2018100100'
                        }
# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/ankittandon',
    'From': 'antand@uw.edu'
}

In [4]:
def api_call(endpoint,parameters):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response

In [5]:
monthly_pageviews_desktop = api_call(endpoint_pageviews, params_pageviews_desktop)

In [6]:
monthly_pageviews_mobile_web = api_call(endpoint_pageviews, params_pageviews_mobile_web)

In [7]:
monthly_pageviews_mobile_app = api_call(endpoint_pageviews, params_pageviews_mobile_app)

In [8]:
monthly_pagecount_desktop = api_call(endpoint_legacy, params_legacy_desktop_site)

In [9]:
monthly_pagecount_mobile = api_call(endpoint_legacy, params_legacy_mobile)

In [10]:
with open('pageviews_desktop-site_201507-201809.json', 'w') as file:
    json.dump(monthly_pageviews_desktop,file)

In [11]:
with open('pageviews_mobile-web_201507-201809.json', 'w') as file:
    json.dump(monthly_pageviews_mobile_web,file)

In [12]:
with open('pageviews_mobile-app_201507-201809.json', 'w') as file:
    json.dump(monthly_pagecount_mobile,file)

In [13]:
with open('pagecount_desktop_200712-201607.json', 'w') as file:
    json.dump(monthly_pagecount_desktop,file)

In [14]:
with open('pagecount_mobile_200712-201607.json', 'w') as file:
    json.dump(monthly_pagecount_mobile,file)

# Using pandas to create a dataframe from the dictionary objects

Every dictionary has a key called 'items' that I will need to index into.

In [15]:
import pandas as pd
pageviews_desktop_df = pd.DataFrame.from_dict(monthly_pageviews_desktop['items'])
pageviews_mobile_web_df = pd.DataFrame.from_dict(monthly_pageviews_mobile_web['items'])
pageviews_mobile_app_df = pd.DataFrame.from_dict(monthly_pageviews_mobile_app['items'])
pagecount_desktop_df = pd.DataFrame.from_dict(monthly_pagecount_desktop['items'])
pagecount_mobile_df = pd.DataFrame.from_dict(monthly_pagecount_mobile['items'])

In order to find the maximum time range available for all datasets, I will concatenate the timestamp field for all datasets and remove duplicates. Finally, I will convert this series to a dataframe. This dataframe will serve as the final dataframe to be output to csv at the end.

In [16]:
combined_timestamp_series = pd.concat([pageviews_desktop_df['timestamp'],pageviews_mobile_web_df['timestamp'],pageviews_mobile_app_df['timestamp'],pagecount_desktop_df['timestamp'],pagecount_mobile_df['timestamp']]).drop_duplicates()
combined_df = combined_timestamp_series.to_frame()

In the next block of code, i'm changing the timestamp field into year and month fields. The first 4 characters in the timestamp are the year and characters 4 to 6 are the month.

In [17]:
combined_df['year']=combined_df['timestamp'].str[0:4]
combined_df['month']=combined_df['timestamp'].str[4:6]

Join the combined dataframe with the pageviews_desktop dataframe on timestamp and select the views column and label it as pageview_desktop_views column. Drop all other extraneous columns from the combined dataframe

In [18]:
combined_df = combined_df.merge(right=pageviews_desktop_df,how='left', on='timestamp')
combined_df['pageview_desktop_views'] = combined_df['views']
combined_df = combined_df.drop(['views','access','agent','granularity','project'],axis=1)

Join the combined dataframe with the pageviews_mobile_app dataframe on timestamp and select the views column and label it as pageview_mobile_app_views column. Drop all other extraneous columns from the combined dataframe

In [19]:
combined_df = combined_df.merge(right=pageviews_mobile_app_df,how='left', on='timestamp', suffixes=('combined','mobile'))
combined_df['pageview_mobile_app_views'] = combined_df['views']
combined_df = combined_df.drop(['access','agent','granularity', 'project','views'],axis=1)

Join the combined dataframe with the pageviews_mobile_web dataframe on timestamp and select the views column and label it as pageview_mobile_web_views column. Drop all other extraneous columns from the combined dataframe

In [20]:
combined_df = combined_df.merge(right=pageviews_mobile_web_df,how='left', on='timestamp', suffixes=('combined','mobile'))
combined_df['pageview_mobile_web_views'] = combined_df['views']
combined_df = combined_df.drop(['access','agent','granularity', 'project','views'],axis=1)

Join the combined dataframe with the pagecount_desktop dataframe on timestamp and select the count column and label it as pagecount_desktop_views column. Drop all other extraneous columns from the combined dataframe

In [21]:
combined_df = combined_df.merge(right=pagecount_desktop_df,how='left', on='timestamp', suffixes=('combined','desktop'))
combined_df['pagecount_desktop_views'] = combined_df['count']
combined_df = combined_df.drop(['access-site','granularity', 'project','count'],axis=1)

Join the combined dataframe with the pagecount_mobile dataframe on timestamp and select the count column and label it as pagecount_mobile_views column. Drop all other extraneous columns from the combined dataframe

In [22]:
combined_df = combined_df.merge(right=pagecount_mobile_df,how='left', on='timestamp', suffixes=('combined','desktop'))
combined_df['pagecount_mobile_views'] = combined_df['count']
combined_df = combined_df.drop(['access-site','granularity', 'project','count'],axis=1)

We are going to create a new column for data with 'all' views and 'all'counts which is a combination of the mobile and web counts and views.

In [23]:
combined_df['pagecount_all_views'] = combined_df['pagecount_desktop_views'] + combined_df['pagecount_mobile_views']
combined_df['pageview_all_views'] = combined_df['pageview_desktop_views'] + combined_df['pageview_mobile_app_views'] + combined_df['pageview_mobile_web_views']

Finally, lets make sure mobile views is a combination of mobile app and mobile web views and let's drop the extraneous columns (mobile app and mobile web views)

In [24]:
combined_df['pageview_mobile_views'] = combined_df['pageview_mobile_app_views'] + combined_df['pageview_mobile_web_views']
combined_df = combined_df.drop(['pageview_mobile_app_views','pageview_mobile_web_views'],axis=1)

Plotting the time series values for page views by medium over time. For sake of visual clariy I am hiding the x axis ticks but have indicated in the title of the plot that the data ranges from December 2007 to September 2018. The number of views is shown at the log scale for the sake of visual clariy also.

In [25]:
import matplotlib.pyplot as plt
plt.plot('timestamp','pageview_desktop_views',data=combined_df)
plt.plot('timestamp','pageview_mobile_views',data=combined_df)
plt.plot('timestamp','pageview_all_views',data=combined_df)
plt.yscale('log')
plt.title("Page Views Metrics over time across medium from December 2007 to September 2018")
plt.xlabel("Timestamp")
plt.ylabel("Number of views")
plt.legend()
plt.tick_params(axis='x',labelbottom=False)
plt.savefig('graph.png')

C:\Users\antand\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\scale.py:111: RuntimeWarning: invalid value encountered in less_equal
  out[a <= 0] = -1000


Finally, I will output the dataframe to csv file so others can view the data. I will not include the index in the output file and also drop the Timestamp column before writing the output.

In [26]:
combined_df = combined_df.drop(['timestamp'],axis=1)
combined_df.to_csv('en-wikipedia_traffic_200712-201809.csv',index=False)